In [11]:
import json
import requests
import pandas as pd

apikey = "vfz0jx0R5n2aE4zGI1ZWs8uBERuCsRvH7PzSYWJL"

print("*** Task 1: Fetch Basic Stock Data ***")
stocks = input("Enter two or more stock symbols separated by commas: ")
url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":stocks}

headers = {
    'x-api-key': apikey
}

response = requests.request("GET", url, headers=headers, params=querystring)
data = response.json()
results = data.get("quoteResponse", {}).get("result", [])

for stock in results:
  ticker = stock.get("symbol")
  company = stock.get("shortName")
  price = stock.get("regularMarketPrice")
  print(f"Stock Ticker: {ticker}, Company: {company}, Current Market Price: ${price}")

*** Task 1: Fetch Basic Stock Data ***
Enter two or more stock symbols separated by commas: AAPL
Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $240.36


In [12]:
print("*** Task 2.1: Fetch Additional Data Using Modules ***")
def call_module(symbol, module):
    url = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}"
    params = {"modules": module}
    resp = requests.get(url, headers=headers, params=params)
    data = resp.json()
    result = data.get("quoteSummary", {}).get("result", [])
    if result:
        return pd.json_normalize(result)
    else:
        return pd.DataFrame()

symbol = input("Enter one stock symbol: ")
module = input("Enter a module from the quoteSummary endpoint: ")

df_summary = call_module(symbol, "summaryDetail")
fiftyTwoWeekHigh = df_summary["summaryDetail.fiftyTwoWeekHigh.fmt"].iloc[0] if "summaryDetail.fiftyTwoWeekHigh.fmt" in df_summary.columns else "N/A"
fiftyTwoWeekLow = df_summary["summaryDetail.fiftyTwoWeekLow.fmt"].iloc[0] if "summaryDetail.fiftyTwoWeekLow.fmt" in df_summary.columns else "N/A"

df_financial = call_module(symbol, "financialData")
returnOnAssets = df_financial["financialData.returnOnAssets.fmt"].iloc[0] if "financialData.returnOnAssets.fmt" in df_financial.columns else "N/A"

df_user = call_module(symbol, module)

data = { "52 Week High": [fiftyTwoWeekHigh], "52 Week Low": [fiftyTwoWeekLow], "Return on Assets (ROA)": [returnOnAssets]}
df = pd.DataFrame(data)

print(df)
print("\n", df_user)

*** Task 2.1: Fetch Additional Data Using Modules ***
Enter one stock symbol: NVDA
Enter a module from the quoteSummary endpoint: assetProfile
  52 Week High 52 Week Low Return on Assets (ROA)
0       153.13       75.61                 59.13%

        assetProfile.address1 assetProfile.city assetProfile.state  \
0  2788 San Tomas Expressway       Santa Clara                 CA   

  assetProfile.zip assetProfile.country assetProfile.phone  \
0            95051        United States     (408) 486-2000   

     assetProfile.website assetProfile.industry assetProfile.industryKey  \
0  https://www.nvidia.com        Semiconductors           semiconductors   

  assetProfile.industryDisp  ... assetProfile.auditRisk  \
0            Semiconductors  ...                      9   

  assetProfile.boardRisk assetProfile.compensationRisk  \
0                     10                             3   

  assetProfile.shareHolderRightsRisk  assetProfile.overallRisk  \
0                                  6

In [13]:
print("*** Task 2.2: Fetch Additional Data Using Modules ***")
trending_url = "https://yfapi.net/v1/finance/trending/US"
response = requests.get(trending_url, headers=headers)
trending_data = response.json()

quotes = trending_data.get("finance", {}).get("result", [])
if quotes:
    quotes = quotes[0].get("quotes", [])
else:
    quotes = []

if not quotes:
    print("No trending stocks data available.")
else:
    trending_list = []
    for stock in quotes:
        ticker = stock.get("symbol")

        quote_url = "https://yfapi.net/v6/finance/quote"
        quote_query = {"symbols": ticker}
        quote_resp = requests.get(quote_url, headers=headers, params=quote_query)
        quote_data = quote_resp.json().get("quoteResponse", {}).get("result", [])

        if quote_data:
            quote_info = quote_data[0]
            name = quote_info.get("shortName")
            current_price = quote_info.get("regularMarketPrice")
            current_price = f"${float(current_price):,.2f}" or "N/A"
        else:
            name = "N/A"
            current_price = "N/A"

        summary_url = f"https://yfapi.net/v11/finance/quoteSummary/{ticker}"
        summary_query = {"modules": "summaryDetail"}
        summary_resp = requests.get(summary_url, headers=headers, params=summary_query)
        summary_data = summary_resp.json()
        summary_result = summary_data.get("quoteSummary", {}).get("result", [])

        if summary_result:
            summary_df = pd.json_normalize(summary_result)
            fiftyTwoWeekHigh = summary_df["summaryDetail.fiftyTwoWeekHigh.fmt"].iloc[0] if "summaryDetail.fiftyTwoWeekHigh.fmt" in summary_df.columns else "N/A"
            fiftyTwoWeekLow = summary_df["summaryDetail.fiftyTwoWeekLow.fmt"].iloc[0] if "summaryDetail.fiftyTwoWeekLow.fmt" in summary_df.columns else "N/A"
        else:
            fiftyTwoWeekHigh = "N/A"
            fiftyTwoWeekLow = "N/A"

        trending_list.append({
            "Ticker": ticker,
            "Name": name,
            "Current Price": current_price,
            "52 Week High": fiftyTwoWeekHigh,
            "52 Week Low": fiftyTwoWeekLow
        })

    trending_df = pd.DataFrame(trending_list)
    print("\nTrending Stocks Data:")
    print(trending_df)

*** Task 2.2: Fetch Additional Data Using Modules ***

Trending Stocks Data:
   Ticker                             Name Current Price 52 Week High  \
0    NVDA               NVIDIA Corporation       $131.28       153.13   
1     CRM                 Salesforce, Inc.       $307.33       369.00   
2    SNOW                   Snowflake Inc.       $166.19       233.88   
3    IONQ                       IonQ, Inc.        $29.93        54.74   
4    BYND                Beyond Meat, Inc.         $3.56      12.1200   
5     APP             Applovin Corporation       $331.00       525.15   
6    EBAY                        eBay Inc.        $69.14        71.61   
7      AI                      C3.ai, Inc.        $26.44        45.08   
8     NIO                         NIO Inc.         $4.72       7.7100   
9    SNPS                   Synopsys, Inc.       $470.46       624.80   
10   MULN          Mullen Automotive, Inc.         $4.04  47,100.0000   
11   NTNX                    Nutanix, Inc.     